In [1]:
# Install Required Packages & Import Dependencies
%%capture
!pip install transformers torch pandas numpy tqdm scikit-learn nltk
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
from typing import Tuple

class MathProblemSolver:
    def __init__(self, model_name="t5-base", checkpoint_path=None):
        self.tokenizer = T5Tokenizer.from_pretrained(model_name)
        self.model = T5ForConditionalGeneration.from_pretrained(model_name)
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.to(self.device)

        if checkpoint_path:
            self.load_checkpoint(checkpoint_path)

        print(f"Using device: {self.device}")

    def load_checkpoint(self, checkpoint_path: str):
        """Load a trained model checkpoint."""
        checkpoint = torch.load(checkpoint_path, map_location=self.device)
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.model.eval()
        print("Loaded trained model checkpoint successfully!")

    def solve(self, problem: str) -> str:
        """Generate equation for a given word problem."""
        self.model.eval()
        input_text = f"Generate equation: {problem}"

        inputs = self.tokenizer(
            input_text,
            max_length=256,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        ).to(self.device)

        with torch.no_grad():
            outputs = self.model.generate(
                input_ids=inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                max_length=256,
                num_beams=5,
                length_penalty=1.0,
                early_stopping=True,
                no_repeat_ngram_size=2,
                do_sample=True,
                top_k=50,
                top_p=0.95,
                temperature=0.7
            )

        equation = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return equation

    def solve_and_compute(self, problem: str) -> Tuple[str, float]:
        """Generate equation and compute the result for a given word problem."""
        equation = self.solve(problem)
        equation_to_solve = equation.split('=')[0].strip()

        try:
            result = eval(equation_to_solve)
            return equation, result
        except:
            return equation, "Unable to compute result"


In [3]:
# Mount Google Drive and Initialize Solver
from google.colab import drive
drive.mount('/content/drive')

# Initialize the solver with your trained model
checkpoint_path = '/content/drive/MyDrive/AWPSS/AWPSS_BEST_MODEL.pt'
solver = MathProblemSolver(checkpoint_path=checkpoint_path)


Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

<ipython-input-1-3b9680cc5fc6>:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=self.device)


Loaded trained model checkpoint successfully!
Using device: cpu


In [5]:
# Interactive Problem Solver
def solve_user_problems():
    while True:
        user_input = input("\nEnter a word problem (or type 'exit' to quit): ")

        if user_input.lower() == 'exit':
            break

        equation, result = solver.solve_and_compute(user_input)
        print(f"\nGenerated equation: {equation}")
        print(f"Result: {result}")

# Run Interactive modes
if __name__ == "__main__":
    print("\nNow starting interactive mode...")
    solve_user_problems()


Now starting interactive mode...

Enter a word problem (or type 'exit' to quit): A pet store had 34 puppies. Later, they received 12 more puppies. How many puppies does the pet store have in total now?

Generated equation: ( 34.0 + 12.0 )
Result: 46.0

Enter a word problem (or type 'exit' to quit): Sarah has 20 marbles in her bag. Her brother gave her 15 more marbles. How many marbles does Sarah have now?

Generated equation: ( 20.0 + 15.0 )
Result: 35.0

Enter a word problem (or type 'exit' to quit): Alex had 50 notebooks. He gave 20 notebooks to his classmates. How many notebooks does Alex have left?

Generated equation: ( 50.0 + 20.0 )
Result: 70.0

Enter a word problem (or type 'exit' to quit): There were 40 books on a shelf. If 18 books were borrowed by students, how many books are left on the shelf?

Generated equation: ( 40.0 - 18.0 )
Result: 22.0

Enter a word problem (or type 'exit' to quit): Each crate holds 8 bottles of juice. If there are 9 crates in total, how many bottle